In [9]:
# IMPORTING ALL THE NECESSARY LIBRARY FOR IMPLIMENTING MODEL

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [11]:
data = pd.read_csv("price.xls")

In [12]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding on the 'admin1' column
data['admin1_encoded'] = label_encoder.fit_transform(data['admin1'])

# Convert to float (if needed)
data['admin1_encoded'] = data['admin1_encoded'].astype(float)

# Apply label encoding on the 'admin1' column
data['category_encoded'] = label_encoder.fit_transform(data['category'])

# Convert to float (if needed)
data['category_encoded'] = data['category_encoded'].astype(float)

# Apply label encoding on the 'admin1' column
data['commodity_encoded'] = label_encoder.fit_transform(data['commodity'])

# Convert to float (if needed)
data['commodity_encoded'] = data['commodity_encoded'].astype(float)

# Apply label encoding on the 'admin1' column
data['market_encoded'] = label_encoder.fit_transform(data['market'])

# Convert to float (if needed)
data['market_encoded'] = data['market_encoded'].astype(float)

In [13]:
new_df = data[['date' , 'latitude' , 'longitude' , 'pricetype' , 'price_inr' , 'usdprice' , 'admin1_encoded' , 'market_encoded' ,'category_encoded' , 'commodity_encoded']]

In [14]:
import pandas as pd

# Assuming new_df is already defined

# Convert the 'date' column to datetime format
new_df['date'] = pd.to_datetime(new_df['date'], errors='coerce')

# Drop rows where 'date' could not be converted (if any)
new_df = new_df.dropna(subset=['date'])

# Extract features
new1_df = new_df.copy()  # Create a new dataframe
new1_df['year'] = new1_df['date'].dt.year
new1_df['month'] = new1_df['date'].dt.month
new1_df['day_of_week'] = new1_df['date'].dt.dayofweek
new1_df['is_weekend'] = new1_df['date'].dt.dayofweek >= 5

# Display the first few rows of the new dataframe
print(new1_df.head())

        date   latitude  longitude  pricetype  price_inr  usdprice  \
0 1994-01-15  28.666667  77.216667          1        8.0    0.2545   
1 1994-01-15  28.666667  77.216667          1        5.0    0.1590   
2 1994-01-15  28.666667  77.216667          1       13.5    0.4294   
3 1994-01-15  28.666667  77.216667          1       31.0    0.9860   
4 1994-01-15  23.033333  72.616667          1        6.8    0.2163   

   admin1_encoded  market_encoded  category_encoded  commodity_encoded  year  \
0             6.0            42.0               0.0               15.0  1994   
1             6.0            42.0               0.0               21.0  1994   
2             6.0            42.0               2.0               17.0  1994   
3             6.0            42.0               3.0                9.0  1994   
4             8.0             3.0               0.0               15.0  1994   

   month  day_of_week  is_weekend  
0      1            5        True  
1      1            5     

C:\Users\gopee\AppData\Local\Temp\ipykernel_21492\731254284.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['date'] = pd.to_datetime(new_df['date'], errors='coerce')


In [ ]:
import pandas as pd
import pmdarima as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Assuming new_df is defined and contains the 'date' column

# Convert 'date' column to datetime format
new_df['date'] = pd.to_datetime(new_df['date'], errors='coerce')

# Drop rows with NaT in the 'date' column
new_df = new_df.dropna(subset=['date'])

# Extract features
new_df['year'] = new_df['date'].dt.year
new_df['month'] = new_df['date'].dt.month
new_df['day_of_week'] = new_df['date'].dt.dayofweek
new_df['is_weekend'] = new_df['date'].dt.dayofweek >= 5

# Check columns in new_df to confirm feature addition
print("Columns in new_df:")
print(new_df.columns)

# Define the target variable 'y'
y = new_df['price_inr']

# Define the exogenous variables 'X'
X = new_df[['admin1_encoded', 'market_encoded', 'category_encoded', 'commodity_encoded', 'year', 'month', 'day_of_week', 'is_weekend']]

# Combine 'y' and 'X' into a single DataFrame to drop missing values
combined_df = pd.concat([y, X], axis=1).dropna()

# Split the combined DataFrame back into 'y' and 'X'
y = combined_df['price_inr']
X = combined_df.drop(columns=['price_inr'])

# Fit the ARIMA model with exogenous variables
auto_arima_model = pm.auto_arima(y, exogenous=X, trace=True, error_action='ignore', suppress_warnings=True)

# Predict the next n_periods (e.g., 10 future periods)
n_periods = 10

# Create a mock X_future DataFrame with the same columns and structure as X
X_future = pd.DataFrame(
    np.zeros((n_periods, X.shape[1])),  # n_periods rows and same number of columns as X
    columns=X.columns
)

# Optionally, fill X_future with more realistic future data if available

# Predict future values
predicted_price_inr = auto_arima_model.predict(n_periods=n_periods, exogenous=X_future)

# Print the predicted 'price_inr'
print("Predicted price_inr values for the next", n_periods, "periods:")
print(predicted_price_inr)

# Assuming you have a test set 'y_test' and predictions 'y_pred' to calculate metrics
# For demonstration purposes, I'll use a portion of 'y' and 'predicted_price_inr'
y_test = y[-n_periods:]  # Last n_periods values from 'y'
y_pred = predicted_price_inr

# Calculate accuracy metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print the accuracy metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


In [15]:
#import pickle

In [16]:
#print(np.__version__)

In [17]:
#print(pd.__version__)

In [18]:
#print(pm.__version__)

In [19]:
#print(sklearn.preprocessing.__version__)

In [20]:
#import sklearn
#print(sklearn.__version__)

## Linear Regression

In [22]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
inputs = new1_df[['admin1_encoded', 'market_encoded', 'category_encoded', 'commodity_encoded', 'year', 'month', 'day_of_week', 'is_weekend']]
targets = new1_df.price_inr

In [23]:
model.fit(inputs , targets)

LinearRegression()

In [24]:
predictions = model.predict(inputs)
predictions

array([120.01000401, 116.60027322, 103.44496202, ...,  70.51117123,
        58.25063097,  54.2726117 ])

## Using Random Forest

In [25]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

In [26]:
inputs = new1_df[['admin1_encoded', 'market_encoded', 'category_encoded', 'commodity_encoded', 'year', 'month', 'day_of_week', 'is_weekend']]
targets = new1_df.price_inr

In [27]:
model.fit(inputs , targets)

RandomForestRegressor()

In [28]:
predictions = model.predict(inputs)
predictions

array([ 8.035 ,  5.037 , 13.71  , ..., 97.8332, 26.2306, 53.2699])

In [29]:
import numpy as np
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(targets, predictions))
print("Root Mean Squared Error:", rmse)


Root Mean Squared Error: 167.08145362869283


In [30]:
model.score(inputs , targets)

0.503582849519614

In [31]:
import pickle

# Assuming 'model' is your trained model
model = RandomForestRegressor()
inputs = new1_df[['admin1_encoded', 'market_encoded', 'category_encoded', 'commodity_encoded', 'year', 'month', 'day_of_week', 'is_weekend']]
targets = new1_df.price_inr
model.fit(inputs, targets)

# Save the model to a file named 'model.pkl'
with open('model1.pkl', 'wb') as file:
    pickle.dump(model, file)